In [93]:
import math
import os
from pathlib import Path

import numpy as np
import librosa as lb
import soundfile as sf
import pyaudio

import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline

import pickle as pkl

from models.VAE_1 import VAE_1
from utils.prepare_data import pad_or_trim

from utils.config import load_config

In [92]:
INVERSE_MFCC_KWARGS = {
    'n_mels': MFCC_KWARGS['n_mels'],
    'dct_type': MFCC_KWARGS['dct_type'],
    'norm': MFCC_KWARGS['norm'],
    'lifter': MFCC_KWARGS['lifter'],
    'n_fft': MFCC_KWARGS['n_fft'],
    'hop_length': MFCC_KWARGS['hop_length'],
    'win_length': MFCC_KWARGS['win_length'],
    'window':  MFCC_KWARGS['window'],
    'center': MFCC_KWARGS['center'],
    'pad_mode': MFCC_KWARGS['pad_mode'],
    'power': MFCC_KWARGS['power'],

    ####
    # 'sr': 22050,
    ####

    'ref': 1.0,
    'n_iter': 32,
    'length': None,
    'dtype': np.float32
}


In [68]:
def play_wave(wave, sr):
    # initialize PyAudio
    p = pyaudio.PyAudio()

    # open a stream
    stream = p.open(format=pyaudio.paFloat32,
                    channels=1,
                    rate=sr,
                    output=True)

    # play audio
    stream.write(wave.tobytes())

    # stop stream and terminate PyAudio
    stream.stop_stream()
    stream.close()
    p.terminate()

In [72]:
def load_random_wave(sample_type, path_to_samples = r"C:\Users\llama\Desktop\cuni\bakalarka\Bachelor_thesis-Electronic_music\data\drums-one_shots"):
    # load random wave
    path = os.path.join(path_to_samples, sample_type, f'{sample_type}_samples')

    sample_name = np.random.choice(os.listdir(path))
    sample, sr = lb.load(os.path.join(path, sample_name))
    return sample, sr

def convert_to_mfcc(wave, sr):
    mfcc = lb.feature.mfcc(y=wave, sr=sr, **MFCC_KWARGS)
    return mfcc

def convert_to_wave(mfcc, sr):
    return lb.feature.inverse.mfcc_to_audio(mfcc=mfcc, sr=sr, **INVERSE_MFCC_KWARGS)

In [86]:
def sample_from_latent_space(mu, logvar):
    std = torch.exp(0.5*logvar)
    eps = torch.randn_like(std)
    return mu + eps*std

def fuse_sampled_samples(samples, weights = None):
    if weights is None:
        weights = np.ones(len(samples)) / len(samples)
    return np.average(samples, weights=weights)

In [94]:
model_dir_path = Path(r'C:\Users\llama\Desktop\cuni\bakalarka\Bachelor_thesis-Electronic_music\trained_models\all_latent-dims=2_noise=multiplicative')
model_name = os.path.basename(model_dir_path)
config = load_config(model_dir_path / f'{model_name}_config.pkl')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\llama\\Desktop\\cuni\\bakalarka\\Bachelor_thesis-Electronic_music\\trained_models\\all_latent-dims=2_noise=multiplicative\\all_latent-dims=2_noise=multiplicative_config.pkl'

In [91]:
wave, sr = load_random_wave('clap')
x = pad_or_trim(convert_to_mfcc(wave, sr), length=100)


reconstructed_x, mu, logvar = model(torch.tensor(x).view(1, 1, *x.shape))

reconstructed_wave = convert_to_wave(reconstructed_x.detach().numpy().reshape(-1, 100), sr)

sampled_x = model.decoder.forward(sample_from_latent_space(mu, logvar))

play_wave(wave, sr)
play_wave(reconstructed_wave, sr)